In [54]:
#dependencies and libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from sklearn.preprocessing import MinMaxScaler
from numpy.random import seed
from tensorflow import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import glob
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score



%matplotlib inline



In [55]:
#import and consolidate CSVs of historical fanatasy football data from 1970 - 2019
root_directory = r"Resources"

all_files = glob.glob(f'{root_directory}\*.csv')

li = []
#apply year to correspnding data using CSV file name
for filename in all_files:
    frame = pd.read_csv(filename, index_col=None, header=0,)
    frame["Year"] = filename[10:14]
    li.append(frame)
#concatinate all 
df = pd.concat(li, axis=0, ignore_index=True)

In [56]:
#validate dataframe
df

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,...,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Year,Tgt
0,0,Ron Johnson,NYG,RB,23.0,14.0,14.0,0.0,0.0,0.0,...,0.0,0.0,1027.0,8.0,263.0,487.0,4.0,261.40,1970,NaN
1,1,Dick Gordon,CHI,WR,26.0,14.0,14.0,0.0,0.0,0.0,...,0.0,0.0,17.0,0.0,4.0,1026.0,13.0,249.30,1970,NaN
2,2,Gene Washington,SFO,WR,23.0,13.0,13.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1100.0,12.0,235.00,1970,NaN
3,3,Gary Garrison,SDG,WR,26.0,14.0,14.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,4.0,1006.0,12.0,217.30,1970,NaN
4,4,MacArthur Lane,STL,RB,28.0,14.0,14.0,0.0,0.0,0.0,...,0.0,0.0,977.0,11.0,206.0,365.0,2.0,240.20,1970,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26123,615,Ray-Ray McCloud,CAR,0,23.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.00,2019,0.0
26124,616,Darrius Shepherd,GNB,WR,24.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.90,2019,2.0
26125,617,Jarrett Stidham,NWE,QB,23.0,3.0,0.0,2.0,4.0,14.0,...,0.0,4.0,-2.0,0.0,2.0,0.0,0.0,-1.64,2019,0.0
26126,618,Michael Walker,JAX,WR,23.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,15.0,0.0,-0.50,2019,3.0


In [57]:
#initiate running back specific dataframe
rbdf=df.loc[df['Pos'] == 'RB']
rbdf  = rbdf.sort_values('G',ascending = False).groupby('Year').head(50)

rbdf

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,...,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Year,Tgt
11903,313,Charles Way,NYG,RB,23.0,16.0,4.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,2.0,76.0,1.0,21.2,1995,12.0
14541,313,Lorenzo Neal,TEN,RB,30.0,16.0,6.0,0.0,0.0,0.0,...,0.0,0.0,-2.0,0.0,1.0,31.0,2.0,23.9,2000,13.0
5862,390,Robert Newhouse,DAL,RB,33.0,16.0,0.0,0.0,0.0,0.0,...,0.0,0.0,34.0,0.0,9.0,0.0,0.0,-2.6,1983,NaN
14592,364,Jerald Sowell,NYJ,RB,26.0,16.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,84.0,0.0,14.4,2000,7.0
5868,396,Rick Berns,RAI,RB,27.0,16.0,0.0,0.0,0.0,0.0,...,0.0,0.0,22.0,0.0,6.0,0.0,0.0,2.2,1983,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5046,34,James Wilder,TAM,RB,24.0,9.0,9.0,0.0,0.0,0.0,...,0.0,0.0,324.0,3.0,83.0,466.0,1.0,146.0,1982,NaN
5037,25,Andra Franklin,MIA,RB,23.0,9.0,9.0,0.0,0.0,0.0,...,0.0,0.0,701.0,7.0,177.0,9.0,0.0,110.0,1982,NaN
5145,133,Charles Alexander,CIN,RB,25.0,9.0,9.0,0.0,0.0,0.0,...,0.0,0.0,207.0,1.0,64.0,85.0,1.0,53.2,1982,NaN
5151,139,Mel Carver,TAM,RB,23.0,9.0,1.0,0.0,0.0,0.0,...,0.0,0.0,229.0,1.0,70.0,46.0,1.0,39.5,1982,NaN


In [58]:
df.Pos.unique()

array(['RB', 'WR', 'QB', 'TE', '0'], dtype=object)

In [59]:
#initiate wide receiver specific dataframe
wrdf=df.loc[df['Pos'] == 'WR']
wrdf  = wrdf.sort_values('G',ascending = False).groupby('Year').head(100)
wrdf


,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,...,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Year,Tgt
25610,102,Emmanuel Sanders,2TM,WR,32.0,17.0,16.0,1.0,1.0,35.0,...,1.0,1.0,0.0,0.0,0.0,869.0,5.0,188.3,2019,97.0
16650,169,Jerry Rice,2TM,WR,42.0,17.0,14.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,429.0,3.0,90.9,2004,64.0
23377,241,Andre Holmes,OAK,WR,27.0,16.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,201.0,4.0,58.1,2015,33.0
8730,16,Anthony Miller,SDG,WR,24.0,16.0,16.0,0.0,0.0,0.0,...,0.0,0.0,21.0,0.0,4.0,1252.0,10.0,260.3,1989,NaN
22428,17,Emmanuel Sanders,DEN,WR,27.0,16.0,16.0,0.0,0.0,0.0,...,0.0,0.0,44.0,0.0,8.0,1404.0,9.0,299.8,2014,141.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,328,Creston Whitaker,NOR,WR,25.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.6,1972,NaN
651,267,Coleman Zeno,NYG,WR,25.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,2.0,97.0,0.0,13.7,1971,NaN
693,309,Gordon Bowdell,DEN,WR,23.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19.0,0.0,2.9,1971,NaN
1926,410,Speedy Thomas,NOR,WR,27.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.3,1974,NaN


In [60]:
#initiate quarterback specific dataframe
qbdf=df.loc[df['Pos'] == 'QB']
qbdf  = qbdf.sort_values('G',ascending = False).groupby('Year').head(25)
qbdf

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,...,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Year,Tgt
19485,78,Jay Cutler,CHI,QB,26.0,16.0,16.0,336.0,555.0,3666.0,...,27.0,555.0,173.0,1.0,40.0,0.0,0.0,223.94,2009,0.0
12713,81,Trent Dilfer,TAM,QB,25.0,16.0,16.0,217.0,386.0,2555.0,...,21.0,386.0,99.0,1.0,33.0,0.0,0.0,178.10,1997,0.0
3517,332,David Humm,OAK,QB,26.0,16.0,0.0,14.0,26.0,151.0,...,0.0,26.0,-4.0,0.0,5.0,0.0,0.0,1.64,1978,NaN
9757,85,Jeff George,IND,QB,24.0,16.0,16.0,292.0,485.0,2910.0,...,10.0,485.0,36.0,0.0,16.0,0.0,0.0,120.00,1991,NaN
9756,84,John Friesz,SDG,QB,24.0,16.0,16.0,262.0,487.0,2896.0,...,12.0,487.0,18.0,0.0,10.0,0.0,0.0,115.64,1991,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5024,12,Ken Anderson,CIN,QB,33.0,9.0,9.0,218.0,309.0,2495.0,...,12.0,309.0,85.0,4.0,25.0,0.0,0.0,160.30,1982,NaN
5336,324,Steve Dils,MIN,QB,27.0,9.0,0.0,11.0,26.0,68.0,...,0.0,26.0,5.0,0.0,1.0,0.0,0.0,3.22,1982,NaN
5144,132,Paul McDonald,CLE,QB,24.0,9.0,3.0,73.0,149.0,993.0,...,5.0,149.0,-13.0,0.0,7.0,0.0,0.0,36.42,1982,NaN
5049,37,Joe Theismann,WAS,QB,33.0,9.0,9.0,161.0,252.0,2033.0,...,13.0,252.0,150.0,0.0,31.0,0.0,0.0,122.32,1982,NaN


In [61]:
#initiate tight end specific dataframe
tedf=df.loc[df['Pos'] == 'TE']
tedf  = tedf.sort_values('G',ascending = False).groupby('Year').head(25)
tedf

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,...,PassingTD,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Year,Tgt
13517,369,Deems May,SEA,TE,29.0,16.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,1.0,9.7,1998,3.0
9503,301,Pete Metzelaars,BUF,TE,30.0,16.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,60.0,1.0,20.0,1990,NaN
9548,346,Scott Galbraith,CLE,TE,23.0,16.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,62.0,0.0,10.2,1990,NaN
9546,344,Pat Carter,RAM,TE,24.0,16.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,58.0,0.0,13.8,1990,NaN
17951,299,Anthony Becht,TAM,TE,29.0,16.0,16.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,115.0,1.0,35.5,2006,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5088,76,Paul Coffman,GNB,TE,26.0,9.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,287.0,2.0,63.7,1982,NaN
5186,174,Joe Rose,MIA,TE,25.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,182.0,2.0,46.2,1982,NaN
5167,155,Mark Brammer,BUF,TE,24.0,9.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,225.0,2.0,59.5,1982,NaN
5178,166,Joe Senser,MIN,TE,26.0,9.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,261.0,1.0,61.1,1982,NaN


# Linear Regression Test For Quarterbacks: PassingTDs and FantasyScore

In [62]:
#plot QB Age to Fantasy Points Scored (top 25)
qbdf25  = qbdf.sort_values('GS',ascending = False).head(25)
qb_plot = qbdf25.hvplot.scatter(
    x="Age",
    y="FantasyPoints",
    title="Fantasy Points by Age"
)
qb_plot

:Scatter   [Age]   (FantasyPoints)

In [63]:
#define X with statistical metrics (age, passing TD. rushing TD, receiving TD, passing yards, rushing yards, receiving yards)
X = qbdf[['Age','PassingTD','PassingYds','RushingTD','RushingYds','ReceivingTD','ReceivingYds']]
X[:5]

,Age,PassingTD,PassingYds,RushingTD,RushingYds,ReceivingTD,ReceivingYds
19485,26.0,27.0,3666.0,1.0,173.0,0.0,0.0
12713,25.0,21.0,2555.0,1.0,99.0,0.0,0.0
3517,26.0,0.0,151.0,0.0,-4.0,0.0,0.0
9757,24.0,10.0,2910.0,0.0,36.0,0.0,0.0
9756,24.0,12.0,2896.0,0.0,18.0,0.0,0.0


In [64]:
#define Y as fanatasy points scored 
y = qbdf[["FantasyPoints"]]
y[:5]

,FantasyPoints
19485,223.94
12713,178.10
3517,1.64
9757,120.00
9756,115.64


In [65]:
#define model as LinearRegression
model = LinearRegression()

In [66]:
#fit model
model.fit(X, y)

LinearRegression()

In [67]:
#print model's slope
print(f"Model's slope: {model.coef_}")

Model's slope: [[0.37867342 4.68788079 0.02814858 5.76685258 0.10263945 6.0463689
  0.23134158]]


In [68]:
#print model's y-intercept
print(f"Model's y-intercept: {model.intercept_}")

Model's y-intercept: [-23.24338471]


In [69]:
#print model's formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

Model's formula: y = [-23.24338471] + [0.37867342 4.68788079 0.02814858 5.76685258 0.10263945 6.0463689
 0.23134158]X


In [70]:
#define predicted y values (Fantasy Points Scored)
predicted_y_values = model.predict(X)

In [71]:
#copy QB predicted 
qbdf_predicted_fantasy_points = qbdf.copy()

In [72]:
#initialize QB predicted points dataframe
qbdf_predicted_fantasy_points["PredictedFantasyPoints"]=predicted_y_values
qbdf_predicted_fantasy_points.head()

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,...,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Year,Tgt,PredictedFantasyPoints
19485,78,Jay Cutler,CHI,QB,26.0,16.0,16.0,336.0,555.0,3666.0,...,555.0,173.0,1.0,40.0,0.0,0.0,223.94,2009,0.0,239.891068
12713,81,Trent Dilfer,TAM,QB,25.0,16.0,16.0,217.0,386.0,2555.0,...,386.0,99.0,1.0,33.0,0.0,0.0,178.10,1997,0.0,172.516721
3517,332,David Humm,OAK,QB,26.0,16.0,0.0,14.0,26.0,151.0,...,26.0,-4.0,0.0,5.0,0.0,0.0,1.64,1978,NaN,-9.557998
9757,85,Jeff George,IND,QB,24.0,16.0,16.0,292.0,485.0,2910.0,...,485.0,36.0,0.0,16.0,0.0,0.0,120.00,1991,NaN,118.330966
9756,84,John Friesz,SDG,QB,24.0,16.0,16.0,262.0,487.0,2896.0,...,487.0,18.0,0.0,10.0,0.0,0.0,115.64,1991,NaN,125.465138


In [73]:
#plot QB predicted fantasy points
qbdf_predicted_fantasy_points25  = qbdf_predicted_fantasy_points.sort_values('G',ascending = False).head(25)
qb_best_fit_line = qbdf_predicted_fantasy_points25.hvplot.scatter(
    x = "Age",
    y = "PredictedFantasyPoints",
    color = "red"
)
qb_best_fit_line

:Scatter   [Age]   (PredictedFantasyPoints)

In [74]:
#combine QB fantasy points by age and best fit
qb_plot * qb_best_fit_line

:Overlay
   .Scatter.I  :Scatter   [Age]   (FantasyPoints)
   .Scatter.II :Scatter   [Age]   (PredictedFantasyPoints)

In [75]:
#testing model accuracy 
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print releveant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.979129863588066.
The r2 is 0.979129863588066.
The mean squared error is 178.3092210580734.
The root mean squared error is 13.353247584691651.
The standard deviation is FantasyPoints    92.432406
dtype: float64.


# Linear Regression Test For Running Backs: PassingTDs and FantasyScore

In [76]:
#plot RB Age to Fantasy Points Scored (top 25)
rbdf25  = rbdf.sort_values('GS',ascending = False).head(25)
rb_plot = rbdf25.hvplot.scatter(
    x="Age",
    y="FantasyPoints",
    title="Fantasy Points by Age"
)
rb_plot

:Scatter   [Age]   (FantasyPoints)

In [77]:
#define X with statistical metrics (age, passing TD. rushing TD, receiving TD, passing yards, rushing yards, receiving yards)
X = rbdf[['Age','PassingTD','PassingYds','RushingTD','RushingYds','ReceivingTD','ReceivingYds']]
X[:5]

,Age,PassingTD,PassingYds,RushingTD,RushingYds,ReceivingTD,ReceivingYds
11903,23.0,0.0,0.0,0.0,6.0,1.0,76.0
14541,30.0,0.0,0.0,0.0,-2.0,2.0,31.0
5862,33.0,0.0,0.0,0.0,34.0,0.0,0.0
14592,26.0,0.0,0.0,0.0,0.0,0.0,84.0
5868,27.0,0.0,0.0,0.0,22.0,0.0,0.0


In [78]:
#define Y as fanatasy points scored 
y = rbdf[["FantasyPoints"]]
y[:5]

,FantasyPoints
11903,21.2
14541,23.9
5862,-2.6
14592,14.4
5868,2.2


In [79]:
#define model as LinearRegression
model = LinearRegression()

#fit model
model.fit(X, y)

#print model metrics (slope, y-intercept, formula)
print(f"Model's slope: {model.coef_}")
print(f"Model's y-intercept: {model.intercept_}")
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

Model's slope: [[ 0.31000318  4.71268254 -0.06593439  5.98282893  0.09814797  5.61590387
   0.20198674]]
Model's y-intercept: [-7.34286068]
Model's formula: y = [-7.34286068] + [ 0.31000318  4.71268254 -0.06593439  5.98282893  0.09814797  5.61590387
  0.20198674]X


In [80]:
#define predicted y values (Fantasy Points Scored)
predicted_y_values = model.predict(X)
#copy RB predicted 
rbdf_predicted_fantasy_points = rbdf.copy()
#initialize QB predicted points dataframe
rbdf_predicted_fantasy_points["PredictedFantasyPoints"]=predicted_y_values
rbdf_predicted_fantasy_points.head()

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,...,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Year,Tgt,PredictedFantasyPoints
11903,313,Charles Way,NYG,RB,23.0,16.0,4.0,0.0,0.0,0.0,...,0.0,6.0,0.0,2.0,76.0,1.0,21.2,1995,12.0,21.342997
14541,313,Lorenzo Neal,TEN,RB,30.0,16.0,6.0,0.0,0.0,0.0,...,0.0,-2.0,0.0,1.0,31.0,2.0,23.9,2000,13.0,19.254336
5862,390,Robert Newhouse,DAL,RB,33.0,16.0,0.0,0.0,0.0,0.0,...,0.0,34.0,0.0,9.0,0.0,0.0,-2.6,1983,NaN,6.224275
14592,364,Jerald Sowell,NYJ,RB,26.0,16.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,84.0,0.0,14.4,2000,7.0,17.684108
5868,396,Rick Berns,RAI,RB,27.0,16.0,0.0,0.0,0.0,0.0,...,0.0,22.0,0.0,6.0,0.0,0.0,2.2,1983,NaN,3.186481


In [81]:
#plot RB predicted fantasy points
rbdf_predicted_fantasy_points25  = rbdf_predicted_fantasy_points.sort_values('G',ascending = False).head(25)
rb_best_fit_line = rbdf_predicted_fantasy_points25.hvplot.scatter(
    x = "Age",
    y = "PredictedFantasyPoints",
    color = "red"
)
rb_best_fit_line

:Scatter   [Age]   (PredictedFantasyPoints)

In [82]:
#combine RB fantasy points by age and best fit
rb_plot * rb_best_fit_line

:Overlay
   .Scatter.I  :Scatter   [Age]   (FantasyPoints)
   .Scatter.II :Scatter   [Age]   (PredictedFantasyPoints)

In [83]:
#testing model accuracy 
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print releveant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.9938831317189012.
The r2 is 0.9938831317189012.
The mean squared error is 45.63893299489922.
The root mean squared error is 6.755659330879498.
The standard deviation is FantasyPoints    86.378007
dtype: float64.


# Linear Regression Test For Wide Receivers: FantasyScore


In [84]:
#plot RB Age to Fantasy Points Scored (top 25)
wrdf25  = wrdf.sort_values('G',ascending = False).head(25)
wr_plot = wrdf25.hvplot.scatter(
    x="Age",
    y="FantasyPoints",
    title="Fantasy Points by Age"
)
wr_plot

:Scatter   [Age]   (FantasyPoints)

In [85]:
#define X with statistical metrics (age, passing TD. rushing TD, receiving TD, passing yards, rushing yards, receiving yards)
X = wrdf[['Age','PassingTD','PassingYds','RushingTD','RushingYds','ReceivingTD','ReceivingYds']]
X[:5]

,Age,PassingTD,PassingYds,RushingTD,RushingYds,ReceivingTD,ReceivingYds
25610,32.0,1.0,35.0,0.0,0.0,5.0,869.0
16650,42.0,0.0,0.0,0.0,0.0,3.0,429.0
23377,27.0,0.0,0.0,0.0,0.0,4.0,201.0
8730,24.0,0.0,0.0,0.0,21.0,10.0,1252.0
22428,27.0,0.0,0.0,0.0,44.0,9.0,1404.0


In [86]:
#define Y as fanatasy points scored 
y = wrdf[["FantasyPoints"]]
y[:5]

,FantasyPoints
25610,188.3
16650,90.9
23377,58.1
8730,260.3
22428,299.8


In [87]:
#define model as LinearRegression
model = LinearRegression()
#fit model
model.fit(X, y)
#print model metrics (slope, y-intercept, formula)
print(f"Model's slope: {model.coef_}")
print(f"Model's y-intercept: {model.intercept_}")
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

Model's slope: [[0.39732278 8.47905274 0.01289346 5.5932665  0.11835292 5.60399599
  0.16787561]]
Model's y-intercept: [-9.6528335]
Model's formula: y = [-9.6528335] + [0.39732278 8.47905274 0.01289346 5.5932665  0.11835292 5.60399599
 0.16787561]X


In [88]:
#define predicted y values (Fantasy Points Scored)
predicted_y_values = model.predict(X)
#copy WR predicted 
wrdf_predicted_fantasy_points = wrdf.copy()
#initialize QB predicted points dataframe
wrdf_predicted_fantasy_points["PredictedFantasyPoints"]=predicted_y_values
wrdf_predicted_fantasy_points.head()

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,...,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Year,Tgt,PredictedFantasyPoints
25610,102,Emmanuel Sanders,2TM,WR,32.0,17.0,16.0,1.0,1.0,35.0,...,1.0,0.0,0.0,0.0,869.0,5.0,188.3,2019,97.0,185.895700
16650,169,Jerry Rice,2TM,WR,42.0,17.0,14.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,429.0,3.0,90.9,2004,64.0,95.865346
23377,241,Andre Holmes,OAK,WR,27.0,16.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,201.0,4.0,58.1,2015,33.0,57.233862
8730,16,Anthony Miller,SDG,WR,24.0,16.0,16.0,0.0,0.0,0.0,...,0.0,21.0,0.0,4.0,1252.0,10.0,260.3,1989,NaN,268.588542
22428,17,Emmanuel Sanders,DEN,WR,27.0,16.0,16.0,0.0,0.0,0.0,...,0.0,44.0,0.0,8.0,1404.0,9.0,299.8,2014,141.0,292.415724


In [89]:
#plot WR predicted fantasy points
wrdf_predicted_fantasy_points25  = wrdf_predicted_fantasy_points.sort_values('G',ascending = False).head(25)
wr_best_fit_line = wrdf_predicted_fantasy_points25.hvplot.scatter(
    x = "Age",
    y = "PredictedFantasyPoints",
    color = "red"
)
wr_best_fit_line

:Scatter   [Age]   (PredictedFantasyPoints)

In [90]:
#combine WR fantasy points by age and best fit
wr_plot * wr_best_fit_line

:Overlay
   .Scatter.I  :Scatter   [Age]   (FantasyPoints)
   .Scatter.II :Scatter   [Age]   (PredictedFantasyPoints)

In [91]:
#testing model accuracy 
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print releveant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.9897925120861412.
The r2 is 0.9897925120861412.
The mean squared error is 65.11464071651538.
The root mean squared error is 8.069364331625843.
The standard deviation is FantasyPoints    79.869302
dtype: float64.


# Linear Regression Test For Tight End: FantasyScore


In [92]:
#plot TE Age to Fantasy Points Scored (top 25)
tedf25  = tedf.sort_values('GS',ascending = False).head(25)
te_plot = tedf25.hvplot.scatter(
    x="Age",
    y="FantasyPoints",
    title="Fantasy Points by Age"
)
te_plot

:Scatter   [Age]   (FantasyPoints)

In [93]:
#define X with statistical metrics (age, passing TD. rushing TD, receiving TD, passing yards, rushing yards, receiving yards)
X = tedf[['Age','PassingTD','PassingYds','RushingTD','RushingYds','ReceivingTD','ReceivingYds']]
X[:5]

,Age,PassingTD,PassingYds,RushingTD,RushingYds,ReceivingTD,ReceivingYds
13517,29.0,0.0,0.0,0.0,0.0,1.0,7.0
9503,30.0,0.0,0.0,0.0,0.0,1.0,60.0
9548,23.0,0.0,0.0,0.0,0.0,0.0,62.0
9546,24.0,0.0,0.0,0.0,0.0,0.0,58.0
17951,29.0,0.0,0.0,0.0,0.0,1.0,115.0


In [94]:
#define Y as fanatasy points scored 
y = tedf[["FantasyPoints"]]
y[:5]

,FantasyPoints
13517,9.7
9503,20.0
9548,10.2
9546,13.8
17951,35.5


In [95]:
#define model as LinearRegression
model = LinearRegression()
#fit model
model.fit(X, y)
#print model metrics (slope, y-intercept, formula)
print(f"Model's slope: {model.coef_}")
print(f"Model's y-intercept: {model.intercept_}")
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

Model's slope: [[0.25920393 4.85886045 0.1514531  3.44449121 0.09416761 6.19032847
  0.17484227]]
Model's y-intercept: [-4.74332975]
Model's formula: y = [-4.74332975] + [0.25920393 4.85886045 0.1514531  3.44449121 0.09416761 6.19032847
 0.17484227]X


In [96]:
#define predicted y values (Fantasy Points Scored)
predicted_y_values = model.predict(X)
#copy TE predicted 
tedf_predicted_fantasy_points = tedf.copy()
#initialize QB predicted points dataframe
tedf_predicted_fantasy_points["PredictedFantasyPoints"]=predicted_y_values
tedf_predicted_fantasy_points.head()

,Unnamed: 0,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,...,PassingAtt,RushingYds,RushingTD,RushingAtt,ReceivingYds,ReceivingTD,FantasyPoints,Year,Tgt,PredictedFantasyPoints
13517,369,Deems May,SEA,TE,29.0,16.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,1.0,9.7,1998,3.0,10.187809
9503,301,Pete Metzelaars,BUF,TE,30.0,16.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,60.0,1.0,20.0,1990,NaN,19.713653
9548,346,Scott Galbraith,CLE,TE,23.0,16.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,62.0,0.0,10.2,1990,NaN,12.058581
9546,344,Pat Carter,RAM,TE,24.0,16.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,58.0,0.0,13.8,1990,NaN,11.618416
17951,299,Anthony Becht,TAM,TE,29.0,16.0,16.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,115.0,1.0,35.5,2006,26.0,29.070773


In [97]:
#plot TE predicted fantasy points
tedf_predicted_fantasy_points25  = tedf_predicted_fantasy_points.sort_values('G',ascending = False).head(25)
te_best_fit_line = tedf_predicted_fantasy_points25.hvplot.scatter(
    x = "Age",
    y = "PredictedFantasyPoints",
    color = "red"
)
te_best_fit_line

:Scatter   [Age]   (PredictedFantasyPoints)

In [98]:
#combine TE fantasy points by age and best fit
te_plot * te_best_fit_line

:Overlay
   .Scatter.I  :Scatter   [Age]   (FantasyPoints)
   .Scatter.II :Scatter   [Age]   (PredictedFantasyPoints)

In [99]:
#testing model accuracy 
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print releveant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.9901660254963087.
The r2 is 0.9901660254963087.
The mean squared error is 29.93516351530995.
The root mean squared error is 5.47130363947295.
The standard deviation is FantasyPoints    55.172959
dtype: float64.
